In [3]:
import torch,os,sys
from torch import nn
import hat.archs.DRCT_arch as DRCT

model = DRCT.DRCT(img_size=64,
                 patch_size=1,
                 in_chans=3,
                 embed_dim=180,
                 depths=(6, 6, 6, 6,6,6,6, 6, 6, 6,6,6),
                 num_heads=(6, 6, 6, 6,6,6,6, 6, 6, 6,6,6),
                 window_size=16,
                 compress_ratio=3,
                 squeeze_factor=30,
                 conv_scale=0.01,
                 overlap_ratio=0.5,
                 mlp_ratio=2.,
                 qkv_bias=True,
                 qk_scale=None,
                 drop_rate=0.,
                 attn_drop_rate=0.,
                 drop_path_rate=0.1,
                 norm_layer=nn.LayerNorm,
                 ape=False,
                 patch_norm=True,
                 use_checkpoint=False,
                 upscale=4,
                 img_range=1.,
                 upsampler='pixelshuffle',
                 resi_connection='1conv',
                 gc = 32)



In [5]:

from fvcore.nn import FlopCountAnalysis, parameter_count_table

# 加载模型

# 定义输入大小
input_tensor = torch.randn(1, 3, 64, 64)

# 计算 FLOPs
flop_count = FlopCountAnalysis(model, input_tensor)
flops = flop_count.total()

# 计算参数量
params_table = parameter_count_table(model)
params = sum(p.numel() for p in model.parameters())

# 计算 Multi-Adds 和前向传播次数
multi_adds = flops / 2
forward_passes = 1

# 输出结果
print(params_table)
print(f"Params: {params}")
print(f"FLOPs: {flops}")
print(f"Multi-Adds: {multi_adds}")
print(f"Forward Passes: {forward_passes}")

/raid/ming/SR24/SR24/hat/archs/DRCT_arch.py:106: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
/raid/ming/SR24/SR24/hat/archs/DRCT_arch.py:182: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, C // self.num_h

| name                             | #elements or shape   |
|:---------------------------------|:---------------------|
| model                            | 14.1M                |
|  conv_first                      |  5.0K                |
|   conv_first.weight              |   (180, 3, 3, 3)     |
|   conv_first.bias                |   (180,)             |
|  patch_embed                     |  0.4K                |
|   patch_embed.norm               |   0.4K               |
|    patch_embed.norm.weight       |    (180,)            |
|    patch_embed.norm.bias         |    (180,)            |
|  layers                          |  13.4M               |
|   layers.0                       |   2.2M               |
|    layers.0.swin1                |    0.3M              |
|    layers.0.adjust1              |    5.8K              |
|    layers.0.swin2                |    0.4M              |
|    layers.0.adjust2              |    6.8K              |
|    layers.0.swin3                |    

In [4]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("CUDA device name:", torch.cuda.get_device_name(0))


CUDA available: True
CUDA device count: 5
CUDA device name: NVIDIA A100-SXM4-40GB


In [4]:
import torch
import torchvision.models as models
from fvcore.nn import FlopCountAnalysis, parameter_count_table
import os 
os.environ['CUDA_VISIBLE_DEVICES']='4'
# 加载模型
# 定义输入大小
input_tensor = torch.randn(1, 3, 64, 64).cuda()  # 将输入张量移动到 GPU

# 将模型移动到 GPU
model = model.cuda()

# 计算 FLOPs
flop_count = FlopCountAnalysis(model, input_tensor)
flops = flop_count.total()

# 计算参数量
params_table = parameter_count_table(model)
params = sum(p.numel() for p in model.parameters())

# 计算 Multi-Adds 和前向传播次数
multi_adds = flops / 2
forward_passes = 1

# 输出结果
print(params_table)
print(f"Params: {params}")
print(f"FLOPs: {flops}")
print(f"Multi-Adds: {multi_adds}")
print(f"Forward Passes: {forward_passes}")

# 释放 GPU 内存
del input_tensor, model
torch.cuda.empty_cache()


/raid/ming/SR24/SR24/hat/archs/DRCT_arch.py:106: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
/raid/ming/SR24/SR24/hat/archs/DRCT_arch.py:182: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, C // self.num_h

RuntimeError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 3.81 GiB total capacity; 2.59 GiB already allocated; 8.44 MiB free; 2.74 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF